### Essaymaker Agent flow

#### Initialize variables

In [4]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv('/Users/felipe_q/Desktop/AI_models/Agents/agent_venv/Agent_Development/langraph/credentials.env'))

True

In [14]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage

memory = SqliteSaver.from_conn_string(":memory:")

### Initialize Agent class

In [ ]:
class AgentState(TypedDict):

    task: str #Human Input.
    plan: str #This will be generated by the planning Agent
    draft: str
    critique: str
    content: List[str] # List of documents that Tavily has researched about.
    revision_number: int #Keepo track of the revisions we have made.
    max_revisions: int #Criteria wheter to stop or not.

In [18]:
# We define our agent
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-5-nano")

### We define the prompts - Each prompt is designed to create different Agents. 1 prompt = 1 Agent.

In [22]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""

RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""

WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""


In [ ]:
## Function to make sure we are getting a list of strings from the language model

from langchain_core.pydantic_v1 import BaseModel #Represents the results that we want to get back from the model.

class Queries(BaseModel):
    queries: List[str] #In this case is a list of strings.

In [ ]:
#Importing the Tavily Client
from tavily import TavilyClient
tavily = TavilyClient()

In [ ]:
# We initialize with the planning agent.

def plan_node(state: AgentState):

    messages = [
        SystemMessage(content = PLAN_PROMPT),
        HumanMessage(content = state['task'])] #The task that we want to do.
    
    response = model.invoke(messages)
    return {'plan': response.content} #We update the plan based on the output of the LLM.
    

#The following function takes the plan of the previous functions and we do some research based on that output

def research_plan_node(state: AgentState):

    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content = RESEARCH_PLAN_PROMPT),
        HumanMessage(content = state['task'])
    ])

    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query = q, max_results =2)
        for r in response['results']:
            content.append(r['content'])
    
    return {'content': content}
    
# Based on the research made in the previous function, we will be generating the first draft.


def generation_node(state: AgentState):
    
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content = f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    
    messages = [
        SystemMessage(
            content = WRITER_PROMPT.format(content = content)
        ),
        user_message]
    
    response = model.invoke(messages)
    return {'draft': response.content, 'revision_number': state.get('revision_number',1)+1}

def reflection_node(state: AgentState):

    messages = [
        SystemMessage(content = REFLECTION_PROMPT),
        HumanMessage(content = state['draft'])
    ]
    response = model.invoke(messages)
    return {'critique': response.content}